In [25]:
import os, sys
import csv
from os.path import join, dirname
import pandas as pd
import numpy as np
from glob import glob 

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import RFE, VarianceThreshold, SelectFromModel
from sklearn.feature_selection import SelectKBest, mutual_info_regression, mutual_info_classif, chi2
from sklearn import metrics
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import KBinsDiscretizer, scale, minmax_scale

In [26]:
root_dir = '/workspace/GradSchool/2020-2/Adv_ML/Final/data'

stat_dir = root_dir + '/epl_player_stat'
result_dir = root_dir + '/epl_result'

In [27]:
result_file = result_dir + '/EPL_Results_from_1993_to_2020.csv'
results = pd.read_csv(result_file)

In [28]:
results = results[['Season', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']]

In [29]:
results.loc[results['HomeTeam'] == 'Man City',"HomeTeam"] = 'Manchester City'
results.loc[results['AwayTeam'] == 'Man City',"AwayTeam"] = 'Manchester City'
results.loc[results['HomeTeam'] == 'Man United',"HomeTeam"] = 'Manchester United'
results.loc[results['AwayTeam'] == 'Man United',"AwayTeam"] = 'Manchester United'
results.loc[results['HomeTeam'] == 'Newcastle',"HomeTeam"] = 'Newcastle United'
results.loc[results['AwayTeam'] == 'Newcastle',"AwayTeam"] = 'Newcastle United'
results.loc[results['HomeTeam'] == 'West Brom',"HomeTeam"] = 'West Bromwich Albion'
results.loc[results['AwayTeam'] == 'West Brom',"AwayTeam"] = 'West Bromwich Albion'
results.loc[results['HomeTeam'] == 'Wolves',"HomeTeam"] = 'Wolverhampton Wanderers'
results.loc[results['AwayTeam'] == 'Wolves',"AwayTeam"] = 'Wolverhampton Wanderers'

In [30]:
is_train = results['Season'].str.contains('2015-16|2016-17|2017-18|2018-19')
is_test = results['Season'].str.contains('2019-20')

In [31]:
train_result = results[is_train]
test_result = results[is_test]

In [32]:
stat1_path = sorted(glob(stat_dir + '/20*.csv'))
stat2_path = sorted(glob(stat_dir + '/pl*.csv'))

In [33]:
train1_stat = pd.DataFrame()
test1_stat = pd.DataFrame()
participating_team = pd.DataFrame(columns=['Season', 'team_name'])

for path in stat1_path:
    
    temp_participating_team = pd.DataFrame(columns=['Season', 'team_name'])
    
    season = path.split('\\')
    season = season[1].split('.')
    season = season[0]
    
    temp = pd.read_csv(path)
    is_epl_player = temp['league'].str.contains('EPL')
    temp = temp[is_epl_player]
    temp.rename(columns={'Unnamed: 0': 'Season'}, inplace=True)
    temp.rename(columns={'player_name': 'Name'}, inplace=True)
    temp['Season'] = season
    
    tmp = temp['teams_played_for']
    for i in range(len(temp)):
        team_name_list = tmp[i].split(',')
        tmp[i] = team_name_list[0]
    temp['teams_played_for'] = tmp
    
    temp_participating_team['team_name'] = list(set(tmp))
    temp_participating_team['Season'] = season
    participating_team = pd.concat([participating_team, temp_participating_team])
    
    
    if season == '2019-20':
        test1_stat = pd.concat([test1_stat, temp])
    else:
        train1_stat = pd.concat([train1_stat, temp])

<ipython-input-33-76b4df548cac>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[i] = team_name_list[0]


In [34]:
train2_stat = pd.DataFrame()
test2_stat = pd.DataFrame()

for path in stat2_path:
    
    season = path.split('\\')
    season = season[1].split('.')
    season = season[0]
    season = season.split('_')
    season = '20'+season[1]
    
    temp = pd.read_csv(path)

    temp.rename(columns={'Unnamed: 0': 'Season'}, inplace=True)
    temp['Season'] = season
    
    tmp = temp['Name']
    for i in range(len(temp)):
        tmp[i] = tmp[i].rstrip()
    temp['Name'] = tmp
    
    if season == '2019-20':
        test2_stat = pd.concat([test2_stat, temp])
    else:
        train2_stat = pd.concat([train2_stat, temp])

<ipython-input-34-c77531e86d95>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp[i] = tmp[i].rstrip()


In [37]:
train2_stat.columns

Index(['Season', 'Name', 'Position', 'Appearances', 'Clean sheets',
       'Goals conceded', 'Tackles', 'Tackle success %', 'Last man tackles',
       'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance',
       'Clearances off line', 'Recoveries', 'Duels won', 'Duels lost',
       'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost',
       'Own goals', 'Errors leading to goal', 'Assists', 'Passes',
       'Passes per match', 'Big chances created', 'Crosses',
       'Cross accuracy %', 'Through balls', 'Accurate long balls',
       'Yellow cards', 'Red cards', 'Fouls', 'Offsides', 'Goals',
       'Headed goals', 'Goals with right foot', 'Goals with left foot',
       'Hit woodwork', 'Goals per match', 'Penalties scored',
       'Freekicks scored', 'Shots', 'Shots on target', 'Shooting accuracy %',
       'Big chances missed', 'Saves', 'Penalties saved', 'Punches',
       'High Claims', 'Catches', 'Sweeper clearances', 'Throw outs',
       'Goal Kicks'],
     

In [11]:
train1_stat = train1_stat[['Season', 'Name', 'teams_played_for', 'minutes_played']]
test1_stat = test1_stat[['Season', 'Name', 'teams_played_for', 'minutes_played']]

In [12]:
train_stat = pd.merge(train2_stat,train1_stat,  how='left', on=['Season', 'Name'])
test_stat = pd.merge(test2_stat,test1_stat,  how='left', on=['Season', 'Name'])

In [13]:
# # stat _col
# ['Season', 'Name', 'Position', 'Appearances', 'Clean sheets', 'Goals conceded', 'Tackles', 'Tackle success %', 'Last man tackles',
#  'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance','Clearances off line', 'Recoveries', 'Duels won', 'Duels lost',
#  'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost', 'Own goals', 'Errors leading to goal', 'Assists', 'Passes',
#  'Passes per match', 'Big chances created', 'Crosses', 'Cross accuracy %', 'Through balls', 'Accurate long balls', 'Yellow cards',
#  'Red cards', 'Fouls', 'Offsides', 'Goals', 'Headed goals', 'Goals with right foot', 'Goals with left foot', 'Hit woodwork',
#  'Goals per match', 'Penalties scored', 'Freekicks scored', 'Shots', 'Shots on target', 'Shooting accuracy %', 'Big chances missed',
#  'Saves', 'Penalties saved', 'Punches', 'High Claims', 'Catches', 'Sweeper clearances', 'Throw outs', 'Goal Kicks']
# ['Season', 'player_name', 'teams_played_for', 'league', 'games', 'minutes_played', 'goals', 'npg', 'assists', 'xG', 'xA',
#  'npxG', 'xG90', 'xA90', 'npxG90', 'position', 'shots', 'key_passes', 'yellow_cards', 'red_cards', 'xGBuildup', 'xGChain']

In [14]:
train_stat = train_stat[['Season', 'Name', 'Position', 'Appearances', 'Clean sheets', 'Goals conceded', 'Tackles', 'Last man tackles',
 'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance','Clearances off line', 'Recoveries', 'Duels won', 'Duels lost',
 'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost', 'Own goals', 'Errors leading to goal', 'Assists',
 'Passes per match', 'Big chances created', 'Crosses', 'Through balls', 'Accurate long balls', 'Yellow cards',
 'Red cards', 'Fouls', 'Offsides', 'Hit woodwork', 'Goals per match', 'Penalties scored', 'Freekicks scored', 'Shots', 
 'Shots on target', 'Big chances missed', 'Saves', 'Penalties saved', 'Punches', 'High Claims', 'Catches', 
 'Sweeper clearances', 'Throw outs', 'Goal Kicks', 'teams_played_for', 'minutes_played']]
test_stat = test_stat[['Season', 'Name', 'Position', 'Appearances', 'Clean sheets', 'Goals conceded', 'Tackles', 'Last man tackles',
 'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance','Clearances off line', 'Recoveries', 'Duels won', 'Duels lost',
 'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost', 'Own goals', 'Errors leading to goal', 'Assists',
 'Passes per match', 'Big chances created', 'Crosses', 'Through balls', 'Accurate long balls', 'Yellow cards',
 'Red cards', 'Fouls', 'Offsides', 'Hit woodwork', 'Goals per match', 'Penalties scored', 'Freekicks scored', 'Shots', 
 'Shots on target', 'Big chances missed', 'Saves', 'Penalties saved', 'Punches', 'High Claims', 'Catches', 
 'Sweeper clearances', 'Throw outs', 'Goal Kicks', 'teams_played_for', 'minutes_played']]

In [15]:
train_stat = train_stat.sort_values(by=['Season','teams_played_for'] ,ascending=True)
train_stat = train_stat.dropna(subset=['teams_played_for'])
train_stat = train_stat.fillna(0)
test_stat = test_stat.sort_values(by=['Season','teams_played_for'] ,ascending=True)
test_stat = test_stat.dropna(subset=['teams_played_for'])
test_stat = test_stat.fillna(0)

In [16]:
change_col_name = ['Clean sheets', 'Goals conceded', 'Tackles', 'Last man tackles',  'Blocked shots', 'Interceptions', 'Clearances', 
             'Headed Clearance','Clearances off line', 'Recoveries', 'Duels won', 'Duels lost', 'Successful 50/50s', 
             'Aerial battles won', 'Aerial battles lost', 'Own goals', 'Errors leading to goal', 'Assists', 
              'Big chances created', 'Crosses', 'Through balls', 'Accurate long balls', 
             'Yellow cards', 'Red cards', 'Fouls', 'Offsides', 'Hit woodwork', 'Penalties scored', 
             'Freekicks scored', 'Shots', 'Shots on target', 'Big chances missed', 'Saves', 'Penalties saved', 'Punches', 
             'High Claims', 'Catches', 'Sweeper clearances', 'Throw outs', 'Goal Kicks', 'minutes_played']

In [17]:
for j in change_col_name:
    train_stat[j] = train_stat[j].astype('float32')
    test_stat[j] = test_stat[j].astype('float32')

In [18]:
# # Convert to stats per game
# for i in train_stat.index:
#     player = train_stat.loc[i]
#     for col in change_col_name:
#         print(col)
#         train_stat[col] = train_stat[col] / player['Appearances']
#         print(train_stat[col])
#         break

# for i in test_stat.index:
#     player = test_stat.loc[i]
#     for col in change_col_name:
#         test_stat[col] = test_stat[col] / player['Appearances']

In [19]:
# train_stat['Appearances'] = train_stat['Appearances'] / 38
# test_stat['Appearances'] = test_stat['Appearances'] / 38

In [20]:
train_stat.to_csv('/workspace/GradSchool/2020-2/Adv_ML/Final/data/processed_data/train_stat.csv', index = False)
test_stat.to_csv('/workspace/GradSchool/2020-2/Adv_ML/Final/data/processed_data/test_stat.csv', index = False)
train_result.to_csv('/workspace/GradSchool/2020-2/Adv_ML/Final/data/processed_data/EPL_Results_from_2014_to_2019.csv', index = False)
test_result.to_csv('/workspace/GradSchool/2020-2/Adv_ML/Final/data/processed_data/EPL_Result_2019-20.csv', index = False)

['Season', 'Name', 'Position', 'Appearances', 'Clean sheets', 'Goals conceded', 'Tackles', 'Last man tackles',
 'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance','Clearances off line', 'Recoveries', 'Duels won', 'Duels lost',
 'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost', 'Own goals', 'Errors leading to goal', 'Assists',
 'Passes per match', 'Big chances created', 'Crosses', 'Through balls', 'Accurate long balls', 'Yellow cards',
 'Red cards', 'Fouls', 'Offsides', 'Hit woodwork', 'Goals per match', 'Penalties scored', 'Freekicks scored', 'Shots', 
 'Shots on target', 'Big chances missed', 'Saves', 'Penalties saved', 'Punches', 'High Claims', 'Catches', 
 'Sweeper clearances', 'Throw outs', 'Goal Kicks', 'teams_played_for', 'minutes_played']

In [21]:
root_dir = '/workspace/GradSchool/2020-2/Adv_ML/Final/data/processed_data'

train_stat_file = root_dir + '/train_stat.csv'
train_result_file = root_dir + '/EPL_Results_from_2014_to_2019.csv'
test_stat_file = root_dir + '/test_stat.csv'
test_result_file = root_dir + '/EPL_Result_2019-20.csv'

train_stat = csv.reader(open(train_stat_file), delimiter=',', quotechar='"')
train_result = csv.reader(open(train_result_file), delimiter=',', quotechar='"')
test_stat = csv.reader(open(test_stat_file), delimiter=',', quotechar='"')
test_result = csv.reader(open(test_result_file), delimiter=',', quotechar='"')

stat_header = next(train_stat)
result_header = next(train_result)

In [22]:
def sum_stat(data, column_list, num, is_team = False):
    value = []
    
    if is_team:
        for i in column_list:
            value.append(data[i].sum())
    else:
        for i in column_list:
            value.append(data[i].sum() / len(data) * 3)
    
    return value

In [23]:
def cal_lineup_value(data, season, team_name, formation = [4,3,3]):
    # data: players stat data list, season: str, team_name:str, formation: list
    team_mem = data[data['teams_played_for'] == team_name]
    team_mem = team_mem[team_mem['Season'] == season]
    
    col = ['Appearances', 'Clean sheets', 'Goals conceded', 'Tackles', 'Last man tackles', 'Blocked shots', 'Interceptions', 'Clearances', 'Headed Clearance','Clearances off line', 'Recoveries', 'Duels won', 'Duels lost', 'Successful 50/50s', 'Aerial battles won', 'Aerial battles lost', 'Own goals', 'Errors leading to goal', 'Assists', 'Passes per match', 'Big chances created', 'Crosses', 'Through balls', 'Accurate long balls', 'Yellow cards', 'Red cards', 'Fouls', 'Offsides', 'Hit woodwork', 'Goals per match', 'Penalties scored', 'Freekicks scored', 'Shots', 'Shots on target', 'Big chances missed', 'Saves', 'Penalties saved', 'Punches', 'High Claims', 'Catches', 'Sweeper clearances', 'Throw outs', 'Goal Kicks', 'minutes_played']
    
    gk_num = 1
    df_num = formation[0]
    mf_num = formation[1]
    fw_num = formation[2]
    
    team_value = pd.DataFrame(columns=col)
    temp_team_value = pd.DataFrame(columns=col)
    
    gk = team_mem[team_mem['Position'] == 'Goalkeeper']    
    df = team_mem[team_mem['Position'] == 'Defender']
    mf = team_mem[team_mem['Position'] == 'Midfielder']
    fw = team_mem[team_mem['Position'] == 'Forward']
    
    # calculate each position's value
    team_gk_value = sum_stat(gk, col,gk_num)
    team_df_value = sum_stat(df, col, df_num)
    team_mf_value = sum_stat(mf, col, mf_num)
    team_fw_value = sum_stat(fw, col, fw_num)
    
    # calculate 11 player's value
    temp_team_value.loc[0] = team_gk_value
    temp_team_value.loc[1] = team_df_value
    temp_team_value.loc[2] = team_mf_value
    temp_team_value.loc[3] = team_fw_value
    
    team_value.loc[0] = sum_stat(temp_team_value, col, 0, is_team=True)
    
    return team_value